In [9]:
#* Import and settings
import os
from os import path
import re

import numpy as np
from scipy.optimize import curve_fit
import sympy as sp
from sympy import Symbol as s
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from lab_tools import graph, lab_round

%matplotlib widget

width = 469.75502 # To get the page width insert "\the\textwidth" in your LaTeX document
graph_path = "../images/"
data_path = "../data/"

mpl.rcParams.update({
    # Show minor ticks
    "ytick.minor.visible": True,
    "xtick.minor.visible": True,
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 11,
    "font.size": 11,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 9,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    # High resolution .png
    "savefig.dpi": 600,
    # Default image size
    "figure.figsize": graph.size(width),
})

In [10]:
RAD_TO_METER = 0.0789 / (2 * np.pi)

In [12]:
raw_data_path = path.join(data_path, "raw")
sorted_data_path = path.join(data_path, "sorted")

shift = 0.081

file_list = [
    file
    for file in os.listdir(raw_data_path
    )
    if re.search(r"[\d\.]+_[\d\.]+_[\d]+_[\w]+", file)
]

for file in file_list:
    match = re.search(r"[\d\.]+_[\d\.]+_(?P<scale>[\d])+_[\w]+", file)
    df = pd.read_csv(
        path.join(raw_data_path, file),
        sep=r"\t",
        engine="python",
    )

    df['da'] = [0.09 / 180 * 2 * np.pi] * len(df.a)
    df['dy'] = df.da * RAD_TO_METER
    df['dI'] = [0.001] * len(df.I)

    df = df[['t', 'I', 'dI', 'a', 'da', 'y', 'dy']]
    
    df.y -= shift + df.y.min()

    df = df.sort_values('t').reset_index(drop=True)
    cut_index = df.y.idxmin()
    df_1, df_2 = df[:cut_index], df[cut_index:]

    df_1 = df_1.sort_values('y').reset_index(drop=True)
    df_2 = df_2.sort_values('y').reset_index(drop=True)

    df_1.to_csv(path.join(sorted_data_path, file.replace(".tsv", "_1.csv")), index=False)
    df_2.to_csv(path.join(sorted_data_path, file.replace(".tsv", "_2.csv")), index=False)